### ドライブのマウント

In [1]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### ライブラリ・モジュールのインポート

In [2]:
# ライブラリの準備
!pip install timm
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from timm.scheduler import CosineLRScheduler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.0 MB/s eta 0:00:00


### シード値の設定

In [3]:
# シード値を設定
def fix_seed(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

fix_seed(seed=1234)

### データセットの準備

In [4]:
# 平均値と標準偏差を計算するためのCIFAR100データセットの前処理を定義
pre_transform_cifar100 = transforms.Compose([
    transforms.ToTensor()
])

# 平均値と標準偏差を計算するためのCIFAR100データセットの読み込み
pre_train_dataset_cifar100 = datasets.CIFAR100(root='/content/data/', download=True, transform=pre_transform_cifar100)

# 平均値と標準偏差を計算するための変数を初期化
pre_mean_cifar100 = 0.0
pre_std_cifar100 = 0.0
pre_total_samples_cifar100 = len(pre_train_dataset_cifar100)

# データセットのすべてのデータポイントに対して平均値と標準偏差を計算
for data in pre_train_dataset_cifar100:
    pre_image, _ = data
    pre_mean_cifar100 += pre_image.mean(dim=(1, 2))  # テンソルのチャンネルごとに平均を計算
    pre_std_cifar100 += pre_image.std(dim=(1, 2))    # テンソルのチャンネルごとに標準偏差を計算

# データセット全体の平均値と標準偏差を計算
pre_mean_cifar100 /= pre_total_samples_cifar100
pre_std_cifar100 /= pre_total_samples_cifar100

print("データセット全体の平均値: ", pre_mean_cifar100)
print("データセット全体の標準偏差: ", pre_std_cifar100)

100%|██████████| 169001437/169001437 [00:03<00:00, 46042668.00it/s]


Extracting /content/data/cifar-100-python.tar.gz to /content/data/
データセット全体の平均値:  tensor([0.5071, 0.4866, 0.4409])
データセット全体の標準偏差:  tensor([0.2009, 0.1984, 0.2023])


In [5]:
# 学習用のCIFAR100データセットの前処理を定義
train_transform_cifar100 = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_cifar100, std=pre_std_cifar100)
    ])
# テスト用のCIFAR100データセットの前処理を定義
test_transform_cifar100 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_cifar100, std=pre_std_cifar100)
    ])

# 学習用のCIFAR100データセットの読み込み
train_dataset_cifar100 = torchvision.datasets.CIFAR100(root='/content/data/', train=True, transform=train_transform_cifar100, download=True)
# テスト用のCIFAR100データセットの読み込み
test_dataset_cifar100 = torchvision.datasets.CIFAR100(root='/content/data/', train=False, transform=test_transform_cifar100, download=True)

# 学習用のCIFAR100データローダーを作成
train_loader_cifar100 = torch.utils.data.DataLoader(dataset=train_dataset_cifar100, batch_size=512, shuffle=True, num_workers=2)
# テスト用のCIFAR100データローダーを作成
test_loader_cifar100 = torch.utils.data.DataLoader(dataset=test_dataset_cifar100, batch_size=512, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


### モデルの実装

In [6]:
# 1×1の畳み込みを定義
def conv1x1(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

# 3×3の畳み込みを定義
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

In [7]:
# Residual Blocksを定義
class BasicBlock(nn.Module):
    # 入力チャンネル数に対する出力チャンネル数の倍率
    expansion = 1

    def __init__(self, in_channels, channels, stride=1):
        super().__init__()
        self.conv1 = conv3x3(in_channels, channels, stride)
        self.bn1 = nn.BatchNorm2d(channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(channels, channels)
        self.bn2 = nn.BatchNorm2d(channels)
        # 入力と出力のチャンネル数が異なる場合（strideが1より大きい場合）、ダウンサンプリング
        if in_channels != channels * self.expansion or stride > 1:
            self.shortcut = nn.Sequential(
                conv1x1(in_channels, channels * self.expansion, stride),
                nn.BatchNorm2d(channels * self.expansion)
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        # 残差写像と恒等写像の和を計算
        out += self.shortcut(x)
        out = self.relu(out)
        return out

In [8]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super().__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, self.in_channels, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # Residual Blocks(in_channels=64)
        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        # Residual Blocks(in_channels=128)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        # Residual Blocks(in_channels=256)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        # Residual Blocks(in_channels=512)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # Heの初期化（正規分布）
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                # 重みを1に初期化
                nn.init.constant_(m.weight, 1)
                # バイアスを0に初期化
                nn.init.constant_(m.bias, 0)

    # Residual Blocksを作成する関数を定義
    def _make_layer(self, block, channels, blocks, stride):
        layers = []
        # 最初のResidual Block（stride=stride）
        layers.append(block(self.in_channels, channels, stride))
        # 残りのResidual Block（stride=1）
        self.in_channels = channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [9]:
# モデルを呼び出す関数を定義
def resnet18_100():
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=100)

In [10]:
# パラメータ数を計算する関数を定義
def num_params(model):
    return sum(x.numel() for x in model.parameters() if x.requires_grad)
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを設定
model = resnet18_100().to(device)
# モデルのパラメータ数を確認
print("ResNet18のパラメータ数: {}".format(num_params(model)))

ResNet18のパラメータ数: 11227812


### 学習と評価（10 epochs）

In [14]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = resnet18_100().to(device)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 10
# warm-upするepoch数を設定
warmup_epochs = 1

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/ResNet18_CIFAR100_10epochs_CLRS_restest.pth')

Epoch [1/10], Train Accuracy: 6.506 %, Loss: 3.9649
Epoch [1/10], Learning Rate: 0.0001
Epoch [2/10], Train Accuracy: 13.466 %, Loss: 3.6850
Epoch [2/10], Learning Rate: 0.001
Epoch [3/10], Train Accuracy: 17.212 %, Loss: 3.1493
Epoch [3/10], Learning Rate: 0.0009779754323328191
Epoch [4/10], Train Accuracy: 24.714 %, Loss: 2.7822
Epoch [4/10], Learning Rate: 0.0009140576474687263
Epoch [5/10], Train Accuracy: 29.368 %, Loss: 2.6094
Epoch [5/10], Learning Rate: 0.000814503363531613
Epoch [6/10], Train Accuracy: 34.014 %, Loss: 2.4776
Epoch [6/10], Learning Rate: 0.0006890576474687264
Epoch [7/10], Train Accuracy: 37.316 %, Loss: 2.3171
Epoch [7/10], Learning Rate: 0.00055
Epoch [8/10], Train Accuracy: 40.686 %, Loss: 2.4793
Epoch [8/10], Learning Rate: 0.0004109423525312737
Epoch [9/10], Train Accuracy: 44.356 %, Loss: 2.0856
Epoch [9/10], Learning Rate: 0.00028549663646838715
Epoch [10/10], Train Accuracy: 46.814 %, Loss: 2.0950
Epoch [10/10], Learning Rate: 0.00018594235253127368


In [15]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 43.38 %


In [16]:
# GPUメモリの解放
del model
torch.cuda.empty_cache()

### 学習と評価（25 epochs）

In [11]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = resnet18_100().to(device)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 25
# warm-upするepoch数を設定
warmup_epochs = 3

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/ResNet18_CIFAR100_25epochs_CLRS_restest.pth')

Epoch [1/25], Train Accuracy: 6.594 %, Loss: 3.9133
Epoch [1/25], Learning Rate: 0.0001
Epoch [2/25], Train Accuracy: 13.978 %, Loss: 3.5205
Epoch [2/25], Learning Rate: 0.00039999999999999996
Epoch [3/25], Train Accuracy: 18.28 %, Loss: 3.3709
Epoch [3/25], Learning Rate: 0.0007
Epoch [4/25], Train Accuracy: 22.462 %, Loss: 2.9074
Epoch [4/25], Learning Rate: 0.001
Epoch [5/25], Train Accuracy: 26.282 %, Loss: 2.7978
Epoch [5/25], Learning Rate: 0.000996451615591515
Epoch [6/25], Train Accuracy: 30.406 %, Loss: 2.6966
Epoch [6/25], Learning Rate: 0.000985862422507884
Epoch [7/25], Train Accuracy: 34.718 %, Loss: 2.3474
Epoch [7/25], Learning Rate: 0.0009683994186497131
Epoch [8/25], Train Accuracy: 37.56 %, Loss: 2.2527
Epoch [8/25], Learning Rate: 0.0009443380060197386
Epoch [9/25], Train Accuracy: 40.528 %, Loss: 2.1562
Epoch [9/25], Learning Rate: 0.0009140576474687263
Epoch [10/25], Train Accuracy: 43.154 %, Loss: 2.0390
Epoch [10/25], Learning Rate: 0.0008780358823396353
Epoch [1

In [12]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 50.96 %


In [13]:
# GPUメモリの解放
del model
torch.cuda.empty_cache()

### 学習と評価（50 epochs）

In [11]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = resnet18_100().to(device)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 50
# warm-upするepoch数を設定
warmup_epochs = 5

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/ResNet18_CIFAR100_50epochs_CLRS_restest.pth')

Epoch [1/50], Train Accuracy: 6.598 %, Loss: 3.9117
Epoch [1/50], Learning Rate: 0.0001
Epoch [2/50], Train Accuracy: 13.748 %, Loss: 3.5239
Epoch [2/50], Learning Rate: 0.00028
Epoch [3/50], Train Accuracy: 18.15 %, Loss: 3.3595
Epoch [3/50], Learning Rate: 0.00045999999999999996
Epoch [4/50], Train Accuracy: 21.928 %, Loss: 2.9469
Epoch [4/50], Learning Rate: 0.0006399999999999999
Epoch [5/50], Train Accuracy: 25.626 %, Loss: 2.8064
Epoch [5/50], Learning Rate: 0.00082
Epoch [6/50], Train Accuracy: 28.868 %, Loss: 2.7775
Epoch [6/50], Learning Rate: 0.001
Epoch [7/50], Train Accuracy: 31.824 %, Loss: 2.3757
Epoch [7/50], Learning Rate: 0.0009991120277927223
Epoch [8/50], Train Accuracy: 35.408 %, Loss: 2.3344
Epoch [8/50], Learning Rate: 0.000996451615591515
Epoch [9/50], Train Accuracy: 38.768 %, Loss: 2.2517
Epoch [9/50], Learning Rate: 0.00099202926282791
Epoch [10/50], Train Accuracy: 41.486 %, Loss: 2.1849
Epoch [10/50], Learning Rate: 0.000985862422507884
Epoch [11/50], Train A

In [12]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 52.46 %


In [13]:
# GPUメモリの解放
del model
torch.cuda.empty_cache()

### 学習と評価（75 epochs）

In [11]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = resnet18_100().to(device)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 75
# warm-upするepoch数を設定
warmup_epochs = 8

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/ResNet18_CIFAR100_75epochs_CLRS_restest.pth')

Epoch [1/75], Train Accuracy: 6.576 %, Loss: 3.9066
Epoch [1/75], Learning Rate: 0.0001
Epoch [2/75], Train Accuracy: 13.82 %, Loss: 3.5107
Epoch [2/75], Learning Rate: 0.00021250000000000002
Epoch [3/75], Train Accuracy: 18.254 %, Loss: 3.3762
Epoch [3/75], Learning Rate: 0.000325
Epoch [4/75], Train Accuracy: 21.746 %, Loss: 2.9389
Epoch [4/75], Learning Rate: 0.0004375
Epoch [5/75], Train Accuracy: 25.454 %, Loss: 2.8922
Epoch [5/75], Learning Rate: 0.00055
Epoch [6/75], Train Accuracy: 28.562 %, Loss: 2.7931
Epoch [6/75], Learning Rate: 0.0006625
Epoch [7/75], Train Accuracy: 31.552 %, Loss: 2.4478
Epoch [7/75], Learning Rate: 0.0007750000000000001
Epoch [8/75], Train Accuracy: 34.1 %, Loss: 2.4383
Epoch [8/75], Learning Rate: 0.0008875
Epoch [9/75], Train Accuracy: 36.388 %, Loss: 2.3713
Epoch [9/75], Learning Rate: 0.001
Epoch [10/75], Train Accuracy: 38.686 %, Loss: 2.2730
Epoch [10/75], Learning Rate: 0.0009996052735444863
Epoch [11/75], Train Accuracy: 40.96 %, Loss: 2.3116
Ep

In [12]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 52.1 %


In [13]:
# GPUメモリの解放
del model
torch.cuda.empty_cache()

### 学習と評価（100 epochs）

In [17]:
# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = resnet18_100().to(device)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/ResNet18_CIFAR100_100epochs_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 6.842 %, Loss: 3.8339
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 14.098 %, Loss: 3.6539
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 18.166 %, Loss: 3.4643
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 22.34 %, Loss: 3.2037
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 26.14 %, Loss: 2.8888
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 28.93 %, Loss: 2.7962
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 31.456 %, Loss: 2.7560
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 34.248 %, Loss: 2.4639
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 36.524 %, Loss: 2.3882
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 38.864 %, Loss: 2.3748
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 40.226 

In [18]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 51.85 %


In [19]:
# GPUメモリの解放
del model
torch.cuda.empty_cache()